In [1]:
!pip install transformers torch stable-baselines3

In [2]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")

def generate_text(prompt, max_length=150):
  response = generator(prompt, max_length=max_length, num_return_sequences=1)
  return response[0]['generated_text']

prompt = "This is sunny day, and"
print(generate_text(prompt))

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


This is sunny day, and there are no children. I had a chance to sit down to talk to Ms. Fieber. The other guests who were there were very interested in my work, and did well. She said that she loved how the work looked, the layout of the rooms, the way people looked at them. They seemed to be the kind that could be trusted to tell her things that they were really not, and there were many ways they could act. A few of them talked about their feelings as people, and many of them told stories that were very different than what is reported in the media. I think that we are in a very different environment now. That will be a long time before we get back to that point


In [3]:
!pip install 'shimmy>=2.0'

In [4]:
import gymnasium as gym
import numpy as np
import torch
from stable_baselines3 import PPO

class ContentFeedbackEnv(gym.Env):
  def __init__(self):
    super(ContentFeedbackEnv, self).__init__()
    self.action_space = gym.spaces.Discrete(3) # 0: 싫어요, 1: 좋아요, 2: 유해 콘텐츠 보고
    self.observation_space = gym.spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
    self.history = []

  def step(self, action):
    if action == 1:
      reward = 1
      feedback = "Like"
    elif action == 2:
      reward = -2
      feedback = "Danger"
    else:
      reward = -1
      feedback = "Hate"

    self.history.append(feedback)

    obs = np.array([0.5])
    terminated = False
    truncated = False
    info = {}

    return obs, reward, terminated, truncated, info

  def reset(self, seed=None, options=None):
    super().reset(seed=seed)
    return np.array([0.5]), {}


In [5]:
env = ContentFeedbackEnv()
model = PPO("MlpPolicy", env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [7]:
past_feedback = [1, 0, 2, 1, 1, 0, 2, 1, 0 ,1]

for action in past_feedback:
  env.step(action)

model.learn(total_timesteps=10000)
model.save('rlhf_content_model')

-----------------------------
| time/              |      |
|    fps             | 565  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 486          |
|    iterations           | 2            |
|    time_elapsed         | 8            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0137306005 |
|    clip_fraction        | 0.356        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.0003       |
|    loss                 | 5.42         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.047       |
|    value_loss           | 61.7         |
------------------------------------------
----------------

In [8]:
model = PPO.load('rlhf_content_model')

env = ContentFeedbackEnv()
model.set_env(env)

prompt = "This is rainy day, so"
response = generate_text(prompt)

print(response)

action = 1 # 0, 2

env.step(action)
model.learn(total_timesteps=10)

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
This is rainy day, so please go to the beach in a nice dark shade with a window for a few minutes."

The two men, who were both wearing a white kimono or a large green jumpsuit, were asked to leave the water. After a few minutes the men were able to find a suitable spot by calling a taxi, but after a minute and a half the cars stopped and there no one came back.

The men stopped in front of the boat, which they were given as a second chance. They were handed cards saying they could leave the boat at any time.

The man was found guilty by a court after his conviction.

The man had previously been given four months to pay the fine
-----------------------------
| time/              |      |
|    fps             | 328  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
